## Exercise 5
In this exercise are we going to apply basic numerical linear algebra methods.  
More specifically, we are going to solve a linear system of equations with a tridiagonal NxN matrix.

#### 1. Gaussian elimination
At first we implement a method to use the Gaussian elimination on a matrix. This is important since it gives us a triangular matrix in row-echelon form.  

If we have a problem of the form Ax=b with A as a matrix, x and b vectors and we're looking for x, we have to apply the same steps of calculation on b as we do on A.  
To do this more efficiently we trianglify a new matrix, which is simply the original matrix A with the vector b added as a new column.

In [2]:
import numpy as np

In [4]:
def triag (B): #B is a matrix composed of A and b
    n = len(B)
    i = 0
    j = 0
    
    while i < n: 
            B[i] = B[i]/B[i][i] #step 1: bring diagonal elements to 1 (apply this operation on the whole row)
            j = i
            while j < n:
                if j == i:
                    j = j+1
                else: 
                    B[j]=B[j]-B[i]*B[j][i] #step 2: elimination of variables 
                    j = j+1
            j = 0
            i = i+1
            y = np.array(B[:,n]).T
            R = np.array(B[:n,:n])
    return R,y,B #R is the matrix in triangular form, y is the result of the calculations applied to b
                 #B is the matrix containing R and b 

#### 2. Backwards substitution
The next step to solve a linear system of equation is the backwards substitution.  
Since this is not needed if the resulting matrix after the Gaussian elimination is a unity matrix, we are going to check that first

In [5]:
unitymatrix = None #define boolean (will later be useful)
def unity(A):
    n = len(A) 
    i = 0
    j = 0
    while i < n:
        if A[i][i] == 1: #check diagonal elements for 1
            while j < n:
                if A[i][j] == 0: #check if all other elements are 0
                    print('unity matrix')
                    j = j+1
                    i = i+1
                    unitymatrix = True
                    break
                else:
                    print('no unity matrix: need for backward substitution')
                    unitymatrix = False
                    break
            break

If we checked our triangular matrix and it is not a unity matrix we need to substitute backwards.  
We will use the algorithm as discussed in the tutorium.  
  
Again, this method uses the linked matrix of matrix and solution.

In [6]:
def backward(C):
    n = len(C)
    m = len(C)-1
    x = np.ones(n) #create an array for the x-vector
    R = np.array(C[:n,:n]) #this is the triangular matrix
    y = np.array(C[:,n]) #this is the solution
    for i in range(m,-1,-1):
        x[i] = y[i]
        for j in range(m,i+1,-1):
            x[i] = x[i] - C[i][j]*x[j]
    return x

#### 3. Tridiagonal matrix and solution
After implementing the different steps to solve a linear system of equations we now want to put it together into one routine.  
  
We are looking at the special case of a NxN tridiagonal matrix.  
Before we can solve the problem, we have to create the matrix, given the values of $\vec{a}$, $\vec{b}$, $\vec{c}$ and $\vec{y}$. 

In [8]:
def matrix(a,b,c,y):
    n = len(b)
    matrix = np.array(np.ones(n)*0) #create a first row with N entries, every entry is set to 0
    i = 1
    mat = matrix
    while i < n:
        mat = np.vstack((mat,matrix)) #iteratively add rows of 0's to the matrix until its form is NxN
        i = i+1
    j = 0
    while j < n:
        mat[j][j] = b[j] #put the b-values on the diagonal
        j = j+1
    k = 0
    while k < n-1:
        mat[k+1][k] = a[k] #put the a-values on the diagonal below
        k = k+1
    p = 0
    while p < n-1:
        mat[p][p+1] = c[p] #put the c- values on the diagonal above b
        p = p+1
    mat = np.hstack((mat,y)) #add the solution-vector y as a new column
    return mat


So the operation chart to solve our problem is:  
1. insert data and create matrix
2. use Gaussian elimination to create a triangular matrix
3. check if this matrix is a unity matrix  
    -> if yes: give out x, this is already the result, no further operations are needed  
    -> if not: use backwards substitution to get the result, give out the new calculated result

In [13]:
def solution(a,b,c,y): #insert all given values
    mat = matrix(a,b,c,y) #create matrix
    R,y,B = triag(mat) #trianglify matrix
    unity(R) #check for unity matrix
    print('the solution is:')
    if unitymatrix == True:
        print(y) #give out solution if it is a unity matrix
    else:
        x = backward(B) #otherwise use backward substitution and return result
    return x    

#### 4. Test the method
In this case we set N to 10, all of the a and c values to -1, the b values to 3 and the solution values to 0.2.  
We will look if the method works and check the result later.

In [14]:
#entry of all of the values as stated above
N = 10
a = np.ones(N-1)*(-1)
b = np.ones(N)*3
c = np.ones(N-1)*(-1)
y = np.array([[0.2],[0.2],[0.2],[0.2],[0.2],[0.2],[0.2],[0.2],[0.2],[0.2]])

#run solution function
solution(a,b,c,y)

no unity matrix: need for backward substitution
the solution is:


array([0.06666667, 0.1       , 0.11428571, 0.12      , 0.12222222,
       0.12307692, 0.12340426, 0.12352941, 0.12357724, 0.12359551])

#### 5. Check the result
Now we will put the calculated values back into our original equation, which was Ax=y.  
Therefore we need the matrix A without the addition of y. To avoid using another function, we will just use *matrix* and slice the row with the y values off.

In [27]:
mat =matrix(a,b,c,y) #create matrix
mat = mat[:10,:10] #slice y off

In [28]:
print(mat)

[[ 3. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  3. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  3. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  3. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  3. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  3. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  3. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  3. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  3. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  3.]]


Since we're multiplying a matrix and a vector we cannot use the normal * to do this. Instead we will use numpys .dot function for matrix multiplication.  
  
The result of this calculation should be an array containing  0.2 as its only value 10 times.

In [31]:
np.dot(mat,y)

array([[0.4],
       [0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.4]])

BLABLABLA VERGLEICH ERGEBNIS BLABLA